---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            if '[edit]' in line:
            # Remember this `state` until the next is found
                state = line
            else:
            # Otherwise, we have a city; keep `state` as last-seen
                university_towns.append((state, line))
    
    university_towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])
    
    #Deleting brackets
    def get_citystate(item):
        if ' (' in item:
            return item[:item.find(' (')]
        elif '[' in item:
            return item[:item.find('[')]
        else:
            return item
     
    university_towns_df =  university_towns_df.applymap(get_citystate)
    
    return university_towns_df

get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel('gdplev.xls', usecols = [4,6], skiprows = 220, header = None)
    GDP.columns = ['Quarter', 'GDP']
    GDP = GDP.set_index('Quarter')
        
    recession_start = None
    for i in range(0,len(GDP['GDP'])-2): 
        if (GDP['GDP'][i] > GDP['GDP'][i+1]) and (GDP['GDP'][i+1] > GDP['GDP'][i+2]):
            recession_start = i+1
            break
            
    return GDP['GDP'].index[recession_start]
    

get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel('gdplev.xls', usecols = [4,6], skiprows = 254, header = None)
    GDP.columns = ['Quarter', 'GDP']
    GDP = GDP.set_index('Quarter')
        
    recession_end = None
    for i in range(0,len(GDP['GDP'])-2):
        if (GDP['GDP'][i] < GDP['GDP'][i+1]) and (GDP['GDP'][i+1] < GDP['GDP'][i+2]):
            recession_end = i+1
            break
            
    return GDP['GDP'].index[recession_end]

get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel('gdplev.xls', usecols = [4,6], skiprows = 254, header = None)
    GDP.columns = ['Quarter', 'GDP']
    GDP = GDP.set_index('Quarter')
    GDP = GDP.loc[get_recession_start():get_recession_end()]
    
    return GDP['GDP'].idxmin() 

get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    quarters = pd.read_excel('gdplev.xls', usecols = [4,6], skiprows = 220, header = None)
    quarters.columns = ['Quarter', 'GDP']
    quarters = quarters.set_index('Quarter')
    quarters = list(quarters.index.values)
    quarters.append('2016q3') 
    
    
    GDP = pd.read_csv('City_Zhvi_AllHomes.csv')
    GDP = GDP.drop(GDP.columns.to_series()['1996-04':'1999-12'], axis=1)
    GDP = GDP.set_index('State')
    GDP['State'] = GDP.index.to_series().map(states)
    GDP = GDP.reset_index(drop=True)
    GDP = GDP.sort(['State', 'RegionName'], ascending=[True, True])
    GDP = GDP.set_index(['State','RegionName'])
    GDP1 = GDP.drop(['RegionID','Metro','CountyName','SizeRank'], axis=1)
    GDP1 = GDP1.groupby(np.arange(len(GDP1.columns))//3, axis=1).mean()
    GDP1.columns = quarters

    
    return GDP1
 
convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #null hipotesis - both type of cities are the same and react in the same way to recession
    #alternative hipotesis - there is difference between city types
    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    
    univ_towns = get_list_of_university_towns()
    univ_towns['univ_town'] = True

    # merge the housing data with university town DataFrames
    regression_data = pd.merge(convert_housing_data_to_quarters(), univ_towns, how='outer', left_index=True, right_on=['State', 'RegionName'])
    regression_data['univ_town'] = regression_data['univ_town'].replace({np.NaN: False})
    
    year_recession_start = int(recession_start[0:4])
    qtr_recession_start = int(recession_start[-1])
    year_recession_bottom = int(recession_bottom[0:4])
    qtr_recession_bottom = int(recession_bottom[-1])

    cols_to_keep = ['State', 'RegionName', 'univ_town']
    qtrs_to_keep = [] #quarters to keep
    for i in range(year_recession_start, year_recession_bottom+1):
        for j in range(1, 5):
            if (i == year_recession_start and j < qtr_recession_start) or (i == year_recession_bottom and j > qtr_recession_bottom):
                pass
            else:
                qtrs_to_keep.append(str(i) + 'q' + str(j))
                
    regression_data = regression_data[cols_to_keep + qtrs_to_keep]
    
    #prices change
    regression_data['price_change'] = regression_data[recession_bottom] - regression_data[recession_start]
    
    #separate prices change to university cities and non-university cities
    univ_town_price_change = regression_data[regression_data['univ_town'] == True]['price_change']
    non_univ_town_price_change = regression_data[regression_data['univ_town'] == False]['price_change']
    
    #T-student test
    t, p = ttest_ind(univ_town_price_change, non_univ_town_price_change, nan_policy='omit',)
    
    #if True -> reject the null hypothesis
    different = False
    if p < 0.01:
        different = True

    #which cities are better
    better = ""
    if univ_town_price_change.mean() > non_univ_town_price_change.mean():
        better = "university town"
    else:
        better = "non-university town"
    
    
    return (different, p, better)

run_ttest()